In [2]:
import pandas as pd
import numpy as np
from datetime import timedelta
import datetime as dt
import plotly.graph_objects as go


import os
import pickle
from config import BASE_DIR
print(os.getcwd())
os.chdir(BASE_DIR)
import scripts.functions as fn 

from scripts.indicators import resample, zigzag, HeikinAshi

from binance.client import Client



c:\Users\lbisa\Mi unidad (leonardo.bisaro@gmail.com)\Cripto\python\trd_app\backtest


In [3]:
#Funcion para descarga de velas
def get_klines(symbol,interval,start,end):
    print(symbol, interval, start, end )
    df = client.get_historical_klines(symbol=symbol, interval=interval, start_str=start, end_str=end)
    df = pd.DataFrame(df)
    df = df.iloc[:, :6]
    df.columns = ["datetime", "open", "high", "low", "close", "volume"]
    df['open'] = df['open'].astype('float')
    df['high'] = df['high'].astype('float')
    df['low'] = df['low'].astype('float')
    df['close'] = df['close'].astype('float')
    df['volume'] = df['volume'].astype('float')
    df['datetime'] = pd.to_datetime(df['datetime'], unit='ms') - dt.timedelta(hours=3)
    #df=df.set_index('datetime')
    return df

In [4]:
environment = 'klines'
#environment = 'binance'

if environment == 'klines':

	klines_file  = './backtest/klines/0m01/Completo_BTCUSDT_0m01_2021-01-01_2023-07-31.DataFrame'
	with open(klines_file, 'rb') as file:
		df = pickle.load(file)
		#df.set_index('datetime',inplace=True)

	df = resample(df,(60*24))
else:
	#Conexion con Binance sin API-KEY
	client = Client()

	start = '2024-05-01'
	end = '2024-11-15'
	symbol = 'BTCUSDT'
	interval = '1d'
	real_start = (dt.datetime.strptime(start, "%Y-%m-%d") ).strftime("%Y-%m-%d")
	real_end = (dt.datetime.strptime(end, "%Y-%m-%d") ).strftime("%Y-%m-%d")
	utc_start = real_start+' 00:00:00'
	utc_end = real_end+' 23:59:59'
	df = get_klines(symbol=symbol,interval=interval, start=utc_start,end=utc_end)


#df = fn.ohlc_mirror_v(df)

#Filtrar rango de fechas
#ini = pd.to_datetime('2018-06-26')
#fin = pd.to_datetime('2028-05-01')
#df = df[(df['datetime']>=ini)&(df['datetime']<=fin)]
#df.reset_index(inplace=True)
#df.drop(columns=['index'],inplace=True)

#df.drop(columns=['volume'],inplace=True)

#Quitar parte del dataframe
#df = df[500:1500]
#df.reset_index(inplace=True)
#df.drop(columns=['index'],inplace=True)

In [10]:
def pivots_acum(df,column):
	last = 0
	for i,r in df.iterrows():
		if r[column] > 0:
			df.at[i,column] = r[column]+last
			last = r[column]+last
		else:
			last = 0
	return df

In [ ]:
df = zigzag(df)

df['price'] = df['close']

pivots = df[df['ZigZag']>0].copy()

#Definir pivots
pivots['side'] = np.where((pivots['ZigZag']>pivots['ZigZag'].shift()),1,-1)
first_pivot_index = pivots.index[0]
pivots.at[first_pivot_index,'side'] = 0 #El primer pivot no se puede definir por lo tanto queda neutro 

#Separar pivots en maximos y minimos
pivots['maxs'] = np.where((pivots['side']>0), pivots['ZigZag'],None)
maxs = pivots[pivots['maxs']>0].copy()
pivots['mins'] = np.where((pivots['side']<0), pivots['ZigZag'],None)
mins = pivots[pivots['mins']>0].copy()

#Acumular maximos crecientes
maxs['maxs_up'] = np.where(maxs['ZigZag']>maxs['ZigZag'].shift(1),1,0)
maxs = pivots_acum(maxs,'maxs_up')

#Acumular minimos crecientes
mins['mins_up'] = np.where(mins['ZigZag']>mins['ZigZag'].shift(1),1,0)
mins = pivots_acum(mins,'mins_up')

#Acumular maximos decrecientes
maxs['maxs_down'] = np.where(maxs['ZigZag']<maxs['ZigZag'].shift(1),1,0)
maxs = pivots_acum(maxs,'maxs_down')

#Acumular minimos decrecientes
mins['mins_down'] = np.where(mins['ZigZag']<mins['ZigZag'].shift(1),1,0)
mins = pivots_acum(mins,'mins_down')

#Incorporar los calculos de maximos y minimos al DF de pivots
pivots['maxs_up'] = None
pivots['maxs_down'] = None
for index, row in maxs.iterrows():
	pivots.at[index,'maxs_up'] = row['maxs_up']
	pivots.at[index,'maxs_down'] = row['maxs_down']
pivots['mins_up'] = None
pivots['mins_down'] = None
for index, row in mins.iterrows():
	pivots.at[index,'mins_up'] = row['mins_up']
	pivots.at[index,'mins_down'] = row['mins_down']


pivots['maxs_up'].ffill(inplace=True)
pivots['maxs_down'].ffill(inplace=True)
pivots['mins_up'].ffill(inplace=True)
pivots['mins_down'].ffill(inplace=True)

pivots['corr_up'] = np.where((
							  ( pivots['ZigZag']>pivots['ZigZag'].shift(1) )
							  & ( pivots['ZigZag']>pivots['ZigZag'].shift(2) )
							  & ( pivots['ZigZag'].shift(1)>pivots['ZigZag'].shift(3) )
							  #& ( pivots['ZigZag'].shift(1)>pivots['ZigZag'].shift(2)*1.02 )
							 ), pivots['close'],None)

pivots['corr_down'] = np.where((
							  ( pivots['ZigZag']<pivots['ZigZag'].shift(1) )
							  & ( pivots['ZigZag']<pivots['ZigZag'].shift(2) )
							  & ( pivots['ZigZag'].shift(1)<pivots['ZigZag'].shift(3) )
							  #& ( pivots['ZigZag'].shift(1)>pivots['ZigZag'].shift(2)*1.02 )
							 ), pivots['close'],None)
df['buy'] = None
df['sell'] = None
df['maxs'] = None
df['mins'] = None
for index, row in pivots.iterrows():
	df.at[index,'buy'] = row['corr_up']
	df.at[index,'sell'] = row['corr_down']
	df.at[index,'maxs'] = row['maxs']
	df.at[index,'mins'] = row['mins']

pivots

C:\Users\lbisa\AppData\Local\Temp\ipykernel_8504\2361716714.py:46: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,datetime,open,high,low,close,volume,psar_low,psar_high,ZigZag,price,...,sell,maxs,mins,side,maxs_up,maxs_down,mins_up,mins_down,corr_up,corr_down
2,2020-06-07,9666.85,9802.00,9372.46,9746.99,57952.848385,NaN,9802.00,9802.0,9746.99,...,None,None,None,0,None,None,None,None,None,None
3,2020-06-08,9746.99,9800.00,9633.00,9782.01,40664.664125,9372.46,NaN,9372.46,9782.01,...,None,None,9372.46,-1,None,None,0,0,None,None
6,2020-06-11,9885.22,9964.00,9113.00,9280.40,94418.984730,NaN,9992.72,9992.72,9280.40,...,None,9992.72,None,1,0,0,None,None,None,None
17,2020-06-22,9294.69,9780.00,9277.09,9685.69,57895.468343,8910.45,NaN,8910.45,9685.69,...,None,None,8910.45,-1,None,None,0,1,None,None
22,2020-06-27,9162.21,9196.24,8833.00,9012.00,46290.930113,NaN,9780.00,9780.0,9012.00,...,None,9780.0,None,1,0,1,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1084,2023-05-25,26329.00,26631.98,25871.89,26473.79,37435.448950,NaN,27495.83,27495.83,26473.79,...,None,27495.83,None,1,0,2,None,None,None,None
1087,2023-05-28,26854.28,28261.32,26764.36,28065.00,43916.008550,25871.89,NaN,25871.89,28065.00,...,None,None,25871.89,-1,None,None,1,0,None,None
1095,2023-06-05,27115.20,27129.33,25388.00,25728.20,65805.603050,NaN,28447.14,28447.14,25728.20,...,None,28447.14,None,1,1,0,None,None,25728.2,None
1109,2023-06-19,26339.98,27068.09,26256.61,26844.35,35872.659740,24800.00,NaN,24800.0,26844.35,...,None,None,24800.0,-1,None,None,0,1,None,None


In [20]:
df_to_show = df #[0:20]
#df_to_show = HeikinAshi(df_to_show)
#df_to_show['close'] = df_to_show['HA_close']
#df_to_show['open'] = df_to_show['HA_open']
#df_to_show['high'] = df_to_show['HA_high']
#df_to_show['low'] = df_to_show['HA_low']
#df_to_show.drop(columns=['price'],inplace=True)
indicators = [
     {'col': 'ZigZag','name': 'ZigZag','color': 'yellow','row': 1, 'mode':'lines',},
     {'col': 'maxs','name': 'maxs','color': 'green','row': 1, 'mode':'markers',},
     {'col': 'mins','name': 'mins','color': 'red','row': 1, 'mode':'markers',},
     #{'col': 'trend-chg','name': 'trend-chg','color': 'yellow','row': 1, 'mode':'markers',},
     #{'col': 'st_high','name': 'ST High','color': 'green','row': 1, 'mode':'markers',},
     #   {'col': 'FL_0.0%', 'name': 'FL_0.0%', 'color': 'red', 'row': 1, 'mode': 'lines', }, 
     #   {'col': 'FL_gold1', 'name': 'FL_gold1', 'color': 'orange', 'row': 1, 'mode': 'lines', }, 
     #   {'col': 'FL_gold2', 'name': 'FL_gold2', 'color': 'orange', 'row': 1, 'mode': 'lines', }, 
     #   {'col': 'FL_100.0%', 'name': 'FL_100.0%', 'color': 'green', 'row': 1, 'mode': 'lines', }, 
        ]

indicators_out = [
     #{'col': 'pred','name': 'pred','color': 'yellow','row': 1, 'mode':'lines',},
     #{'col': 'VOL_perc','name': 'Volume_SMA','color': 'green','row': 1, 'mode':'lines',},
        ]

#https://plotly.com/python/reference/scatter/#scatter-marker-symbol
events = [
       #{'df':df_to_show,'col':'buy','name': 'BUY','color': 'green','symbol': 'triangle-up' }, 
       #{'df':df_to_show,'col':'sell','name': 'SELL','color': 'red','symbol': 'triangle-down' }, 
     ]


fig = fn.ohlc_chart(df_to_show,indicators=indicators,indicators_out=indicators_out,events=events,) 
fig.show()
